In [1]:
import pandas as pd 
import numpy as np
import yfinance as yf

#### Getting the data

In [9]:
symbol = input("Enter the symbol you want to perform DCF on: ")

In [10]:
ticker = yf.Ticker(symbol)

In [11]:
Balance_sheet = ticker.balance_sheet.transpose()
Income_statement = ticker.income_stmt.transpose()

In [13]:
actual = pd.DataFrame(columns=sorted(list(Income_statement.index.year)))
forecast = pd.DataFrame(columns=[x + max(list(Income_statement.index.year)) for x in range(1,6)]).transpose()

In [67]:
Balance_sheet['Other Asset'] = Balance_sheet['Other Current Assets'] + Balance_sheet['Other Non Current Assets'] + Balance_sheet['Other Intangible Assets']

In [71]:
# all these are in other liabilities becase they will have the same growth in our model
Balance_sheet['Other liabilities'] = Balance_sheet['Non Current Deferred Taxes Liabilities'] +Balance_sheet['Pensionand Other Post Retirement Benefit Plans Current'] 
Balance_sheet['Other liabilities'] = Balance_sheet['Other Current Liabilities'] + Balance_sheet['Other Non Current Liabilities']

#### BS assumptions

##### Modeling Trade receivables

In [29]:
actual_receivables = actual.copy()
actual_receivables = (Balance_sheet['Gross Accounts Receivable'] / Income_statement['Operating Revenue']) * 360
actual_receivables = pd.DataFrame(actual_receivables, columns=['Days receivables']).sort_index()
actual_receivables.index = actual_receivables.index.year
actual_receivables

,Days receivables
2020,82.562249
2021,83.086479
2022,81.514299
2023,83.815115


In [40]:
forecast_receivables = forecast.copy()
forecast_revenue = pd.read_csv('Data/Forecast_Revenue.csv').set_index('Unnamed: 0', drop=True)
forecast_receivables['Forecast receivables'] = (forecast_revenue['forecast Revenues'] * actual_receivables['Days receivables'].iloc[-1]) / 360
forecast_receivables

,Forecast receivables
2024,5.630614e+10
2025,6.425841e+10
2026,7.333381e+10
2027,8.369094e+10
2028,9.551085e+10


##### Modeling Trade Payables

In [41]:
actual_payables = actual.copy()
actual_payables = (Balance_sheet['Accounts Payable'] / Income_statement['Cost Of Revenue']) * 360
actual_payables = pd.DataFrame(actual_payables, columns=['Days Payables']).sort_index()
actual_payables.index = actual_payables.index.year
actual_payables

,Days Payables
2020,97.894874
2021,104.508347
2022,109.177973
2023,98.905303


In [43]:
forecast_payables = forecast.copy()  
forecast_Cogs = pd.read_csv('Data/forecast_COGS.csv').set_index('Unnamed: 0', drop=True)
forecast_payables['Forecast Payables'] = (forecast_Cogs['forecast COGS'] * actual_payables['Days Payables'].iloc[-1]) / 360
forecast_payables

,Forecast Payables
2024,2.092498e+10
2025,2.388028e+10
2026,2.725295e+10
2027,3.110196e+10
2028,3.549458e+10


##### Modeling Inventory

In [48]:
actual_inventory = actual.copy()
actual_inventory = (Balance_sheet['Inventory'] / Income_statement['Cost Of Revenue']) * 360
actual_inventory = pd.DataFrame(actual_inventory, columns=['Days Inventory']).sort_index()
actual_inventory.index = actual_inventory.index.year
actual_inventory

,Days Inventory
2020,14.80533
2021,18.168173
2022,21.502314
2023,13.664728


In [46]:
forecast_inventory = forecast.copy()  
forecast_Cogs = pd.read_csv('Data/forecast_COGS.csv').set_index('Unnamed: 0', drop=True)
forecast_inventory['Forecast Payables'] = (forecast_Cogs['forecast COGS'] * actual_inventory['Days Inventory'].iloc[-1]) / 360
forecast_inventory

,Forecast Payables
2024,2.890990e+09
2025,3.299292e+09
2026,3.765260e+09
2027,4.297038e+09
2028,4.903921e+09


##### Modeling PP&E

In [66]:
actual_PPE = actual.copy()
actual_PPE = Balance_sheet['Net PPE']
actual_PPE = pd.DataFrame(actual_PPE).rename({'Net PPE' : 'PP&E'}, axis=1)
actual_PPE['% of revenue'] = actual_PPE['PP&E'] / Income_statement['Operating Revenue']
actual_PPE.index = actual_PPE.index.year
actual_PPE

,PP&E,% of revenue
2023,109987000000.0,0.519015
2022,87546000000.0,0.441549
2021,70803000000.0,0.421226
2020,52904000000.0,0.369919


In [75]:
forecast_PPE = forecast.copy()
forecast_PPE['Forecast PP&E'] = actual_PPE['% of revenue'].mean() * forecast_revenue['forecast Revenues']
forecast_PPE

,Forecast PP&E
2024,1.059102e+11
2025,1.208682e+11
2026,1.379388e+11
2027,1.574203e+11
2028,1.796532e+11


##### Modeling Other Assets

In [72]:
actual_oth_asset = Balance_sheet['Other Asset']
actual_oth_asset = pd.DataFrame(actual_oth_asset)
actual_oth_asset['% of revenue'] = actual_oth_asset['Other Asset'] / Income_statement['Operating Revenue']
actual_oth_asset

,Other Asset,% of revenue
2023-06-30,61774000000.0,0.291504
2022-06-30,50119000000.0,0.252782
2021-06-30,36268000000.0,0.215768
2020-06-30,31658000000.0,0.221361


In [76]:
forecast_oth_asset = forecast.copy()
forecast_oth_asset['Forecast Other asset'] = actual_oth_asset['% of revenue'].mean() * forecast_revenue['forecast Revenues']
forecast_oth_asset

,Forecast Other asset
2024,5.933739e+10
2025,6.771777e+10
2026,7.728174e+10
2027,8.819645e+10
2028,1.006527e+11


##### Modeling Other liabilities

In [74]:
actual_oth_liabilities = Balance_sheet['Other liabilities']
actual_oth_liabilities = pd.DataFrame(actual_oth_liabilities)
actual_oth_liabilities['% of revenue'] = actual_oth_liabilities['Other liabilities'] / Income_statement['Operating Revenue']
actual_oth_liabilities

,Other liabilities,% of revenue
2023-06-30,32726000000.0,0.15443
2022-06-30,28593000000.0,0.144212
2021-06-30,25093000000.0,0.149285
2020-06-30,20659000000.0,0.144453


In [77]:
forecast_oth_liabilities = forecast.copy()
forecast_oth_liabilities['Forecast other liabilities'] = actual_oth_liabilities['% of revenue'].mean() * forecast_revenue['forecast Revenues']
forecast_oth_liabilities

,Forecast other liabilities
2024,3.581597e+10
2025,4.087436e+10
2026,4.664716e+10
2027,5.323527e+10
2028,6.075383e+10
